### Data used: https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [99]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math

In [100]:
with open("../data/shakespeare.txt", "r", encoding="utf-8") as f:
    text = f.read()

In [101]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [102]:
print("".join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


### Encoder and Decoder section

#### Further development idea:
Use Sentencepiece (https://github.com/google/sentencepiece) or Tiktoken (https://github.com/openai/tiktoken) intead of the following mapping

In [103]:
stoi = {
    ch:i for i, ch in enumerate(chars)
    }

itos = {
    i:ch for i, ch in enumerate(chars)
    }

In [104]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [105]:
print(encode("Encode this"))
print(decode(encode("Encode this")))

[17, 52, 41, 53, 42, 43, 1, 58, 46, 47, 57]
Encode this


#### Encoding the entire dataset

In [106]:
data = torch.tensor(encode(text), dtype=torch.long)

In [107]:
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [108]:
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [109]:
block_size = 8
batch_size = 4
train_data[:block_size + 1]

torch.manual_seed = 9876

In [110]:
def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i + 1:i+block_size + 1] for i in ix])

    return x, y

In [111]:
xb, yb = get_batch("train")

In [112]:
class BiagramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets = None): #B, T, C = Batch, Time, Channel
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets) #Pytorch cross_entropy expects B, T and C in this way when dealing with multi dimensional structures
        
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

In [113]:
model = BiagramLanguageModel(vocab_size)
logits, loss = model(xb, yb)

print(logits.shape)

torch.Size([32, 65])


In [116]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=100)[0].tolist()))


TeGHLZcWM$E&jD
-pGi&dhy-CgcV3jKqNv!H?:rirxer?-j -nLRW:nHdtfoovb&&-j-!WMNgCdFE nLra
RIGf.fVSJoNFtrakA


In [114]:
print(loss)

tensor(4.5398, grad_fn=<NllLossBackward0>)


### Negative log likelyhood

In [115]:
math.log(1/vocab_size) * -1

4.174387269895637

Expecting ~4.17 loss

In [122]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")

    #Evaluating the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.578493118286133


#### Let's check the improvement

In [123]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(model.generate(idx, max_new_tokens=500)[0].tolist()))


And s s f ol he'tharekllaletommy ilin, t s wemiruthe onoller, at any mertloufuldsturedibofou an tsos


#### No numbers is a good start